# Part 3: 메모리 추가 💾

## 🎯 목적
세션 간에도 사용자 정보를 유지하는 **영구 상태 관리**를 추가합니다.

### Before
- 세션이 바뀌면 이전 정보 소실

### After
- 사용자별로 중요한 정보를 저장하고 재사용

### 핵심 개념
- **💾 Checkpointer**: 대화 상태 저장/복원
- **🏷️ Thread ID**: 세션 식별자
- **🗂️ Persistent State**: 누적 이력 기반 컨텍스트

In [ ]:
# 필요한 라이브러리 임포트
from typing import List, Optional
from datetime import datetime
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
import uuid

## 메모리 추출기 설정

대화에서 중요한 정보를 자동으로 추출하여 구조화된 형태로 저장합니다.

In [ ]:
# Pydantic 모델 정의
class MemoryItem(BaseModel):
    """개별 메모리 아이템"""
    key: str = Field(description="메모리 키 (예: user_name, preference, fact)")
    value: str = Field(description="메모리 값")
    category: str = Field(
        description="카테고리 (personal_info, preference, interest, relationship, fact, etc.)"
    )
    importance: int = Field(description="중요도 (1-5, 5가 가장 중요)", ge=1, le=5)
    confidence: float = Field(description="추출 신뢰도 (0.0-1.0)", ge=0.0, le=1.0)


class ExtractedMemories(BaseModel):
    """추출된 메모리 컬렉션"""
    memories: List[MemoryItem] = Field(description="추출된 메모리 아이템 리스트")
    summary: str = Field(description="대화 내용 요약")
    timestamp: str = Field(
        default_factory=lambda: datetime.now().isoformat(), 
        description="추출 시간"
    )

In [ ]:
def create_memory_extractor(model: str = "gpt-4o"):
    """메모리 추출기를 생성합니다."""
    
    # Output Parser 생성
    memory_parser = PydanticOutputParser(pydantic_object=ExtractedMemories)
    
    # 시스템 프롬프트
    system_prompt = """You are an expert memory extraction assistant. 
    Extract important information from user conversations and convert them into structured key-value pairs.
    
    Extract ALL relevant information including:
    - Personal information (name, age, location, occupation, etc.)
    - Preferences and interests
    - Important facts or events mentioned
    - Goals and aspirations"""
    
    # 프롬프트 템플릿
    template = f"""{system_prompt}
    
    User Input: {{input}}
    
    {{format_instructions}}
    """
    
    prompt = ChatPromptTemplate.from_template(
        template,
        partial_variables={
            "format_instructions": memory_parser.get_format_instructions()
        },
    )
    
    # 모델 설정
    llm = ChatOpenAI(model=model, temperature=0)
    
    # 메모리 추출 체인 생성
    memory_extractor = prompt | llm | memory_parser
    
    return memory_extractor

## 메모리 기능이 있는 그래프 구성

In [ ]:
from typing import Any
from langchain_core.runnables import RunnableConfig
from langgraph.graph import StateGraph, MessagesState, START
from langgraph.store.base import BaseStore
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o", temperature=0)
memory_extractor = create_memory_extractor(model="gpt-4o")


def call_model(
    state: MessagesState,
    config: RunnableConfig,
    *,
    store: BaseStore,
) -> dict[str, Any]:
    """LLM 모델을 호출하고 사용자 메모리를 관리합니다.
    
    Args:
        state: 현재 상태 (메시지 포함)
        config: 실행 설정
        store: 메모리 저장소
    """
    # 마지막 메시지에서 user_id 추출
    user_id = config["configurable"]["user_id"]
    namespace = ("memories", user_id)
    
    # 유저의 메모리 검색
    memories = store.search(namespace, query=str(state["messages"][-1].content))
    info = "\n".join([f"{memory.key}: {memory.value}" for memory in memories])
    system_msg = f"You are a helpful assistant. User info: {info}" if info else "You are a helpful assistant."
    
    # 사용자가 기억 요청 시 메모리 저장
    last_message = state["messages"][-1]
    if "remember" in last_message.content.lower():
        result = memory_extractor.invoke({"input": str(state["messages"][-1].content)})
        for memory in result.memories:
            print(f"💾 저장: {memory.key} = {memory.value}")
            store.put(namespace, str(uuid.uuid4()), {memory.key: memory.value})
    
    # LLM 호출
    response = model.invoke(
        [{"role": "system", "content": system_msg}] + state["messages"]
    )
    return {"messages": response}

In [ ]:
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.store.memory import InMemoryStore

# 그래프 빌드
builder = StateGraph(MessagesState)
builder.add_node("call_model", call_model)
builder.add_edge(START, "call_model")

# 메모리 체크포인터 생성
# 실제 프로덕션에서는 PostgresSaver 사용 권장
memory_saver = InMemorySaver()
memory_store = InMemoryStore()

# 그래프 컴파일
graph_with_memory = builder.compile(
    checkpointer=memory_saver,
    store=memory_store,
)

print("✅ 메모리 기능이 있는 그래프 생성 완료!")

### 메모리 테스트

In [ ]:
def run_graph(msg, thread_id="default", user_id="default"):
    """그래프를 실행하는 헬퍼 함수"""
    config = {
        "configurable": {
            "thread_id": thread_id + user_id,
            "user_id": user_id,
        }
    }
    print(f"\n[유저🙋] {msg}")
    
    result = graph_with_memory.invoke(
        {"messages": [{"role": "user", "content": msg}]},
        config=config,
    )
    
    print(f"[AI🤖] {result['messages'][-1].content}")
    return result

In [ ]:
# 메시지, thread_id, user_id 전달
run_graph("안녕? 내 이름은 테디야", "1", "someone")

In [ ]:
# 같은 thread에서 대화 계속
run_graph("내 이름이 뭐라고?", "1", "someone")

In [ ]:
# 다른 thread에서는 기억하지 못함
run_graph("내 이름이 뭐라고?", "2", "someone")

### 🧠 장기 기억 저장: `remember` 키워드

메시지에 `remember` 가 포함되면 중요 정보를 장기 저장소에 기록합니다.

In [ ]:
# remember 키워드로 장기 기억에 저장
run_graph("내 이름이 테디야 remember", "2", "someone")

In [ ]:
# 추가 정보 저장
run_graph(
    "내 직업은 AI Engineer 야. 내 취미는 Netflix 보기 야. remember", 
    "4", 
    "someone"
)

In [ ]:
# 다른 스레드에서도 장기 기억은 유지됨
run_graph("내 이름, 직업, 취미 알려줘", "100", "someone")

In [ ]:
# 다른 user_id로 실행한 경우는 기억하지 못함
run_graph("내 이름, 직업, 취미 알려줘", "100", "other")

### 📊 State 확인

현재 저장된 상태를 조회해 메시지 이력과 체크포인트 정보를 확인합니다.

In [ ]:
# 임의의 Config 설정
config = {
    "configurable": {
        "thread_id": "100" + "someone",
        "user_id": "someone",
    }
}

# 현재 상태 가져오기
snapshot = graph_with_memory.get_state(config)

print("📊 현재 상태 정보:")
print(f"- 메시지 수: {len(snapshot.values['messages'])}개")
print(f"- 체크포인트 ID: {snapshot.config['configurable']['checkpoint_id']}")

# 최근 메시지 몇 개 표시
print("\n[최근 메시지]")
for msg in snapshot.values["messages"][-3:]:
    role = msg.type if hasattr(msg, "type") else "unknown"
    content = msg.content if hasattr(msg, "content") else str(msg)
    print(f"  [{role}]: {content[:100]}..." if len(content) > 100 else f"  [{role}]: {content}")

---

# Part 4: Human-in-the-Loop 🙋

## 🎯 목적
고위험/중요 작업에 대해 AI가 스스로 멈추고 인간 승인을 요청하는 승인 기반 흐름을 도입합니다.

### 언제 승인이 필요한가
- **💰 금융 처리**: 결제/이체/투자
- **🏥 의료 조언**: 처방/치료 권고
- **📧 대외 커뮤니케이션**: 공지/발송
- **🔐 보안 변경**: 권한/설정

### 핵심 개념
- **⏸️ interrupt**: 실행 일시정지 및 승인 대기
- **📋 Command**: 승인/거부 후 재개 명령
- **💡 Human Approval**: 승인 워크플로우(검토 → 결정 → 재개)

In [ ]:
from langchain_core.tools import tool
from langgraph.types import Command, interrupt

@tool
def human_assistance(query: str) -> str:
    """Request assistance from an expert(human)."""
    # interrupt를 호출하여 실행 일시 중지
    # 사람의 응답을 기다림
    human_response = interrupt({"query": query})
    
    # 사람의 응답 반환
    return human_response["data"]

### 🗺️ HITL 그래프 구성

`human_assistance` 도구를 통해 승인 지점을 구현합니다.

- **역할**: 필요 시 interrupt 로 중단 후 인간 답변 대기
- **흐름**: chatbot → tools(`human_assistance`) → chatbot → END

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import InMemorySaver

# State 정의
class State(TypedDict):
    messages: Annotated[list, add_messages]

# 도구 리스트
tools_with_human = [human_assistance]

# 새로운 그래프 구성
graph_builder_hitl = StateGraph(State)

# LLM에 도구 바인딩
llm = ChatOpenAI(model="gpt-4o", temperature=0)
llm_with_human_tools = llm.bind_tools(tools_with_human)


def chatbot_with_human(state: State):
    """Human Interruption 요청할 수 있는 챗봇"""
    message = llm_with_human_tools.invoke(state["messages"])
    
    # interrupt 중 병렬 도구 호출 방지
    # (재개 시 도구 호출이 반복되는 것을 방지)
    if hasattr(message, "tool_calls"):
        assert (
            len(message.tool_calls) <= 1
        ), "병렬 도구 호출은 interrupt와 함께 사용할 수 없습니다"
    
    return {"messages": [message]}


# 노드 추가
graph_builder_hitl.add_node("chatbot_with_human", chatbot_with_human)

# ToolNode 추가
tool_node_hitl = ToolNode(tools=tools_with_human)
graph_builder_hitl.add_node("tools", tool_node_hitl)

# 엣지 추가
graph_builder_hitl.add_conditional_edges("chatbot_with_human", tools_condition)
graph_builder_hitl.add_edge("tools", "chatbot_with_human")
graph_builder_hitl.add_edge(START, "chatbot_with_human")

# 메모리와 함께 컴파일
memory_hitl = InMemorySaver()
graph_hitl = graph_builder_hitl.compile(checkpointer=memory_hitl)

print("✅ Human-in-the-Loop 그래프 생성 완료!")

### 🎬 HITL 테스트

사람에게 조언을 요청하는 질문으로 interrupt 와 재개 흐름을 검증합니다.

In [ ]:
from langchain_core.messages import HumanMessage
from langchain_core.runnables import RunnableConfig

# 인간 지원을 요청하는 메시지
user_input = "LangGraph가 뭐야? 사람한테 듣고 싶어."
config_hitl = {"configurable": {"thread_id": str(uuid.uuid4())}}

print(f"User: {user_input}\n")

# 실행 (interrupt에서 중단될 것임)
try:
    result = graph_hitl.invoke(
        {"messages": [HumanMessage(content=user_input)]},
        config=config_hitl,
    )
except Exception as e:
    print(f"⏸️ 실행이 중단되었습니다: {e}")

In [ ]:
# 상태 확인 - 어느 노드에서 중단되었는지 확인
snapshot = graph_hitl.get_state(config_hitl)
print(f"\n📊 현재 상태:")
print(f"  다음 실행할 노드: {snapshot.next}")
print(f"  체크포인트 ID: {snapshot.config['configurable']['checkpoint_id']}")

In [ ]:
# 인간의 응답으로 실행 재개
human_response = """## 전문가의 조언:
LangGraph는 LangChain 팀에서 개발한 프레임워크로, 상태 기반 AI 애플리케이션을 
그래프 구조로 구현할 수 있게 해줍니다. 주요 특징:
- 상태 관리와 체크포인트
- 조건부 분기와 순환 구조
- Human-in-the-Loop 지원
- 도구 통합과 멀티 에이전트 시스템
"""

# Command 객체로 재개
human_command = Command(resume={"data": human_response})

print(f"\n💡 사람의 응답: {human_response[:100]}...\n")

# 재개
result = graph_hitl.invoke(human_command, config=config_hitl)
print(f"\n[최종 응답]\n{result['messages'][-1].content}")

## 📚 Part 3-4 요약

### Part 3: 메모리 추가
1. **Checkpointer**: 대화 상태 영구 저장
2. **Thread ID**: 세션별 대화 컨텍스트 관리
3. **장기 기억**: `remember` 키워드로 중요 정보 저장
4. **Store**: 사용자별 프로필 정보 관리

### Part 4: Human-in-the-Loop
1. **interrupt**: 실행 중단 및 인간 개입 대기
2. **Command**: 승인/거부 후 재개 명령
3. **도구 기반 승인**: `human_assistance` 도구로 구현
4. **병렬 호출 제한**: interrupt 시 단일 도구만 호출

### 실무 활용 시나리오
- **고객 서비스 봇**: 고객 정보 기억 및 민감한 요청 승인
- **의료 상담 봇**: 환자 이력 관리 및 처방 전 의사 승인
- **금융 어시스턴트**: 거래 이력 저장 및 고액 거래 승인
- **교육 도우미**: 학습 진도 추적 및 중요 결정 시 교사 승인

### 주의사항
- **개인정보 보호**: 민감한 정보는 암호화하여 저장
- **승인 정책**: 명확한 승인 기준과 에스컬레이션 규칙 정의
- **감사 추적**: 모든 승인/거부 이력 기록
- **타임아웃 처리**: interrupt 상태에서 장시간 대기 시 처리 방안